# KOFIC 일별 박스오피스  

https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do

해당 사이트에 접속하여 엑셀 데이터를 다운받는다

# 1. 라이브러리

In [78]:
import time
import os
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException
from glob import glob

# 2. 동적 웹크롤링을 통한 엑셀 데이터 다운로드

## 추출 범위 지정

다운받다 도중에 오류로 인해 중지되면 해당 날짜부터 다시 진행할 수 있도록 시작 기준 날짜 지정

In [ ]:
# 1. 크롤링 시작 기준 날짜
## 날짜는 일주일씩 전주로 변경되어 탐색이 이루어진다
START_DATE = "2025-09-06"

# 2. 몇 주간의 데이터를 다운로드할지 설정
## KOFIC의 날짜 기간은 최대 일주일이기 때문에
NUM_WEEKS = 52*30 # 약 30년치

# 3. KOBIS 일별 박스오피스 URL
URL = "https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do"

## 웹크롤링 함수 제작

by Gemini

In [ ]:
def download_boxoffice_data():
    # 웹 드라이버 설정
    DOWNLOAD_FOLDER_PATH = "D:\\공모전\\25-2 비어플\\영화관람객크롤링\\엑셀데이터"  #파일 다운로드 경로 설정
    # 다운로드 대기 최대 시간(초)
    DOWNLOAD_TIMEOUT = 120 #사이트 로딩이 오래 걸릴때가 있다

    # 크롬 브라우저 옵션
    chrome_options = webdriver.ChromeOptions()
    prefs = {"download.default_directory": DOWNLOAD_FOLDER_PATH}
    chrome_options.add_experimental_option("prefs", prefs)

    # GoogleBot으로 설정하여 혹시나 있을 ip벤을 막는다
    googlebot_user_agent = "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"
    chrome_options.add_argument(f"user-agent={googlebot_user_agent}")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(URL)
    print(f"페이지 접속 완료: {URL}")
    print(f"파일 다운로드 경로: {DOWNLOAD_FOLDER_PATH}")

    # current_date는 조회 종료일을 의미
    ## 조회 시작일을 지정하는 것보다 종료일을 지정하는 것이 오류가 안난다
    current_date = datetime.strptime(START_DATE, "%Y-%m-%d")

    try:
        for i in range(NUM_WEEKS):
            # 1. 조회 기간 계산
            # current_date를 조회 종료일로 설정
            end_date = current_date
            # 조회 시작일을 종료일로부터 6일 전으로 설정 (일주일)
            start_date = end_date - timedelta(days=6)

            # 날짜를 문자열로 변환
            end_date_str = end_date.strftime("%Y-%m-%d")
            start_date_str = start_date.strftime("%Y-%m-%d")
            
            print("-" * 30)
            print(f"({i+1}/{NUM_WEEKS}) 작업 시작: {start_date_str} ~ {end_date_str}")

            # 2. #sSearchTo 태그에 조회 종료일 값 입력
            date_to_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#sSearchTo"))
            )
            driver.execute_script("arguments[0].value = arguments[1];", date_to_input, end_date_str)
            print(f"조회 종료일 입력: {end_date_str}")

            # 3. #sSearchFrom 태그에 조회 시작일 값 입력
            date_from_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#sSearchFrom"))
            )
            driver.execute_script("arguments[0].value = arguments[1];", date_from_input, start_date_str)
            print(f"조회 시작일 입력: {start_date_str}")
            
            # 4. '검색' 버튼 클릭
            search_button = driver.find_element(By.CSS_SELECTOR, "#searchForm > div > div.wrap_btn > button")
            search_button.click()
            print("검색 버튼 클릭 완료")

            # '엑셀' 버튼을 기다리는 동안 StaleElementReferenceException이 발생하면
            # 오류를 내지 않고 목표 요소가 나타날 때까지 다시 시도
            wait = WebDriverWait(driver, 10, ignored_exceptions=[StaleElementReferenceException])
            download_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, "//a[text()='엑셀']"))
            )


            
            # 다운로드 전, 기존에 남아있을 수 있는 파일을 먼저 정리
            ## 모든 파일은 데이터를 다운받는 날짜 (이 경우는 9월 7일)로 저장된다
            ### 아래 8번에 있는 파일이름 변경 로직과 연계한다
            default_filename = "KOBIS_일별_박스오피스_2025-09-07.xls"
            target_filepath = os.path.join(DOWNLOAD_FOLDER_PATH, default_filename)
            if os.path.exists(target_filepath):
                os.remove(target_filepath)

            # 5. '엑셀' 버튼 클릭
            download_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[text()='엑셀']"))
            )
            download_button.click()
            print("엑셀 버튼 클릭!")

            # 6. 다운로드 확인 알림창(Alert) 처리
            WebDriverWait(driver, 10).until(EC.alert_is_present())
            alert = driver.switch_to.alert
            alert.accept()
            print("다운로드 확인 창 '확인' 클릭 완료. 파일 다운로드를 시작합니다.")

            # 7. 다운로드가 완료될 때까지 대기
            start_time = time.time()
            while True:
                # .crdownload 파일이 없으면 다운로드가 끝난 것으로 간주
                if not glob(os.path.join(DOWNLOAD_FOLDER_PATH, '*.crdownload')):
                    # 다운로드된 최종 파일이 존재하는지 한 번 더 확인
                    if os.path.exists(target_filepath):
                        print("파일 다운로드 완료.")
                        break
                
                # 최대 대기 시간을 초과하면 오류를 발생시키고 종료
                if time.time() - start_time > DOWNLOAD_TIMEOUT:
                    raise TimeoutError("파일 다운로드 시간이 너무 오래 걸립니다.")
                
                # 0.5초 간격으로 반복적인 확인
                time.sleep(0.5)
                

            # 8. 파일 이름 변경
            ## 조회 시작일과 종료일로 이름을 변경한다
            ### 중간에 오류가 발생하였을 시 해당 기간부터 시작할 수 있게 하기 위함이다

            new_filename = f"KOBIS_{start_date_str}_to_{end_date_str}.xls"
            new_filepath = os.path.join(DOWNLOAD_FOLDER_PATH, new_filename)
            os.rename(target_filepath, new_filepath)
            print(f"파일 이름 변경 완료: {new_filename}")

            time.sleep(0.5) #다음 날짜 이동 전 잠시 대기

            # 9. 다음 조회를 위해 날짜를 1주일 전으로 이동
            current_date -= timedelta(days=7)

    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
    finally:
        print("-" * 30)
        print("모든 작업이 완료되었습니다. 5초 후에 브라우저를 종료합니다.")
        time.sleep(5)
        driver.quit()

## 웹크롤링 함수 시작

In [81]:
download_boxoffice_data()

페이지 접속 완료: https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do
파일 다운로드 경로: D:\공모전\25-2 비어플\영화관람객크롤링\엑셀데이터
------------------------------
(1/1560) 작업 시작: 2003-11-02 ~ 2003-11-08
조회 종료일 입력: 2003-11-08
조회 시작일 입력: 2003-11-02
검색 버튼 클릭 완료
엑셀 버튼 클릭!
다운로드 확인 창 '확인' 클릭 완료. 파일 다운로드를 시작합니다.
파일 다운로드 완료.
파일 이름 변경 완료: KOBIS_2003-11-02_to_2003-11-08.xls
------------------------------
(2/1560) 작업 시작: 2003-10-26 ~ 2003-11-01
조회 종료일 입력: 2003-11-01
조회 시작일 입력: 2003-10-26
검색 버튼 클릭 완료
엑셀 버튼 클릭!
다운로드 확인 창 '확인' 클릭 완료. 파일 다운로드를 시작합니다.
파일 다운로드 완료.
파일 이름 변경 완료: KOBIS_2003-10-26_to_2003-11-01.xls
------------------------------
(3/1560) 작업 시작: 2003-10-19 ~ 2003-10-25
조회 종료일 입력: 2003-10-25
조회 시작일 입력: 2003-10-19
검색 버튼 클릭 완료
엑셀 버튼 클릭!
다운로드 확인 창 '확인' 클릭 완료. 파일 다운로드를 시작합니다.
파일 다운로드 완료.
파일 이름 변경 완료: KOBIS_2003-10-19_to_2003-10-25.xls
------------------------------
(4/1560) 작업 시작: 2003-10-12 ~ 2003-10-18
조회 종료일 입력: 2003-10-18
조회 시작일 입력: 2003-10-12
검색 버튼 클릭 완료
엑셀 버튼 클릭!
다운로드 확인 창 '확인' 클릭 완료. 파일 다운로드